In [1]:
import pandas as pd
import os
import collections

In [2]:
mode_dict = {0: 'Other', 1:'Walk',2:'Bike',3:'SOV',4:'HOV2',5:'HOV3+',6:'Transit',8:'School Bus'}
annualization_factor = 300

In [3]:
# Relative path between notebooks and goruped output directories
relative_path = '../../../outputs'
output_vision_file = os.path.join(relative_path, 'Vision2050_longlist.csv')

In [4]:
# Load network results
network_df = pd.read_excel(os.path.join(relative_path,'network/') + r'network_summary_detailed.xlsx',
                  sheetname='Network Summary')

In [5]:
# Load Daysim results
trip = pd.read_csv(os.path.join(relative_path,r'daysim/') + r'_trip.tsv', sep='\t')
person = pd.read_csv(os.path.join(relative_path,r'daysim/') + r'_person.tsv', sep='\t')

In [6]:
# Average HBW/ Non-HBW Trip Distance
trip['Trip Type'] = 'Not Home-Based Work'
trip.ix[(((trip['opurp']==0) & (trip['dpurp']==1)) | ((trip['opurp']==1) & (trip['dpurp']==0))),'Trip Type']= 'Home-Based Work'
hbw_trips = trip.loc[trip['Trip Type']=='Home-Based Work']
nhbw_trips = trip.loc[trip['Trip Type']=='Not Home-Based Work']

output_dict = {'Average Commute Trip Length':hbw_trips['travdist'].mean()}
output_dict = collections.OrderedDict(output_dict)
output_dict['Average Other Trip Length']=nhbw_trips['travdist'].mean()

In [7]:
# VMT per person per day
network_df['all_facility_vmt'] = network_df['arterial_vmt']+ network_df['connectors_vmt']+ network_df['highway_vmt']
total_vmt = network_df['all_facility_vmt'].sum()
total_persons = person.sum()['psexpfac']
vmt_per_person = total_vmt/total_persons
output_dict['Daily VMT per person']=vmt_per_person

In [8]:
# VHT per person per day
network_df['all_facility_vht'] = network_df['arterial_vht']+ network_df['connectors_vht']+ network_df['highway_vht']
total_vht = network_df['all_facility_vht'].sum()
vht_per_person = total_vht/total_persons
output_dict['Daily VHT per person']= vht_per_person

In [9]:
# Delay per person annually
network_df['all_facility_delay'] = network_df['arterial_delay']+ network_df['connectors_delay']+ network_df['highway_delay']
total_delay = network_df['all_facility_delay'].sum()
delay_per_person = (total_delay/total_persons)*annualization_factor
output_dict['Annual Delay Hours per Person']= vht_per_person

In [10]:
# Total Delay Hours Daily by Facility Type
df_fac = pd.DataFrame(network_df.sum()[['arterial_delay','highway_delay']])
df_fac = df_fac.reset_index()
df_fac.columns = ['Facility Type', 'Delay']
#df_fac.index = df_fac['Facility Type']
#df_fac.drop('Facility Type', axis=1, inplace=True)
df_fac.loc['Total'] = df_fac.sum()
output_dict['Daily Arterial Delay Hours'] = df_fac['Delay'].loc[df_fac['Facility Type'] == 'arterial_delay'].values[0]
output_dict['Daily Highway Delay Hours'] = df_fac['Delay'].loc[df_fac['Facility Type'] == 'highway_delay'].values[0]

In [11]:
# Daily Transit Boardings
transit_df = pd.read_excel(r'../../../outputs/network/network_summary_detailed.xlsx', sheetname='Transit Summaries')
tod_list = ['5to6','6to7','7to8','8to9','9to10','10to14','14to15','15to16','16to17','17to18','18to20']
transit_df = transit_df[[tod+'_board' for tod in tod_list]+['route_code']]
transit_df = transit_df.fillna(0)
transit_df['line_total'] = transit_df[[tod+'_board' for tod in tod_list]].sum(axis=1)
output_dict['Daily Transit Boardings']=transit_df['line_total'].sum()
output_df = pd.DataFrame(output_dict.items(), columns = ['Data Item', 'Value'])
output_df['Group'] = 'Total'

In [12]:
# All Trip Mode Share
model_df = trip[['mode','trexpfac']].groupby('mode').sum()[['trexpfac']]/trip[['trexpfac']].sum()
model_df = model_df.reset_index()

model_df.replace({'mode':mode_dict}, inplace=True)
model_df.columns = ['Data Item', 'Value']
model_df['Group'] = 'All Trip Mode Share'

output_df =pd.concat([output_df, model_df])

In [13]:
# Commute Trip Mode Share
model_df_hbw = hbw_trips[['mode','trexpfac']].groupby('mode').sum()[['trexpfac']]/trip[['trexpfac']].sum()
model_df_hbw = model_df_hbw.reset_index()
model_df_hbw.replace({'mode':mode_dict}, inplace=True)
model_df_hbw.columns = ['Data Item', 'Value']
model_df_hbw['Group'] = 'Commute Trip Mode Share'

output_df =pd.concat([output_df, model_df_hbw])

In [14]:
# Non-Commute Trip Mode Share
model_df_nhbw = nhbw_trips[['mode','trexpfac']].groupby(['mode']).sum()[['trexpfac']]/nhbw_trips[['trexpfac']].sum()
model_df_nhbw.reset_index(inplace=True)
model_df_nhbw.replace({'mode':mode_dict}, inplace=True)
model_df_nhbw.columns = ['Data Item', 'Value']
model_df_nhbw['Group'] = 'Non-Commute Trip Mode Share'

output_df =pd.concat([output_df, model_df_nhbw])


In [15]:
# Number and Percent of People Walking, Biking, or Transiting
trip_person = pd.merge(trip,person,on=['hhno','pno'], how='left')
bike_walk_t_trips = trip_person[trip_person['mode'].isin([1,2,6])]

df = bike_walk_t_trips.groupby(['hhno','pno']).count()
df = df.reset_index()
df = df[['hhno','pno']]
df['bike_walk_t'] = True

df = pd.merge(person,df,on=['hhno','pno'], how='left')
df['bike_walk_t'] = df['bike_walk_t'].fillna(False)

df_share = pd.DataFrame(df.groupby('bike_walk_t').sum()['psexpfac']/df['psexpfac'].sum())
df_total =  pd.DataFrame(df.groupby('bike_walk_t').sum()['psexpfac'])
bike_walk_dict= {'Share of People Walking, Biking, and Using Transit':df_share.loc[True]['psexpfac'], 
                  'Total People Walking, Biking, and Using Transit':df_total.loc[True]['psexpfac']}
bike_walk_df = pd.DataFrame(bike_walk_dict.items(), columns = ['Data Item', 'Value'])

bike_walk_df['Group'] = 'Total'
output_df =pd.concat([output_df, bike_walk_df])


In [16]:
output_df.to_csv(output_vision_file, index= False)